<h1> Getting started with TensorFlow </h1>

In this notebook, you play around with the TensorFlow Python API.

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


<h2> Adding two tensors </h2>

First, let's try doing this using numpy, the Python numeric package. numpy code is immediately evaluated.

In [2]:
a = np.array([5, 3, 8])
b = np.array([3, -1, 2])
c = np.add(a, b)
print(c)

[ 8  2 10]


The equivalent code in TensorFlow consists of two steps:
<p>
<h3> Step 1: Build the graph </h3>

In [3]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.add(a, b)
print(c)

Tensor("Add:0", shape=(3,), dtype=int32)


c is an Op ("Add") that returns a tensor of shape (3,) and holds int32. The shape is inferred from the computation graph.

Try the following in the cell above:
<ol>
<li> Change the 5 to 5.0, and similarly the other five numbers. What happens when you run this cell? </li>
<li> Add an extra number to a, but leave b at the original (3,) shape. What happens when you run this cell? </li>
<li> Change the code back to a version that works </li>
</ol>

<p/>
<h3> Step 2: Run the graph

In [4]:
with tf.Session() as sess:
  result = sess.run(c)
  print(result)

[ 8  2 10]


<h2> Using a feed_dict </h2>

Same graph, but without hardcoding inputs at build stage

In [5]:
a = tf.placeholder(dtype=tf.int32, shape=(None,))  # batchsize x scalar
b = tf.placeholder(dtype=tf.int32, shape=(None,))
c = tf.add(a, b)
with tf.Session() as sess:
  result = sess.run(c, feed_dict={
      a: [3, 4, 5],
      b: [-1, 2, 3]
    })
  print(result)

[2 6 8]


<h2> Heron's Formula in TensorFlow </h2>

The area of triangle whose three sides are $(a, b, c)$ is $\sqrt{s(s-a)(s-b)(s-c)}$ where $s=\frac{a+b+c}{2}$ 

Look up the available operations at https://www.tensorflow.org/api_docs/python/tf

In [3]:
triangle_sides = [[5,3,7], [2.1,4.4,5.4], [2.1,4.4,5.4]]

In [7]:
def compute_area(sides):
  a = sides[:, 0]
  b = sides[:, 1]
  c = sides[:, 2]
  
  s = (a+b+c)/2
  
  return tf.sqrt(s*(s-a)*(s-b)*(s-c))

In [8]:
with tf.Session() as sess:
  area = compute_area(tf.cast(tf.constant(triangle_sides), tf.float32))
  
  result = sess.run(area)
  print(result)

[6.4951906 4.419121  4.419121 ]


<h2> Placeholder and feed_dict </h2>

More common is to define the input to a program as a placeholder and then to feed in the inputs. The difference between the code below and the code above is whether the "area" graph is coded up with the input values or whether the "area" graph is coded up with a placeholder through which inputs will be passed in at run-time.

In [9]:
with tf.Session() as sess:
  sides = tf.placeholder(tf.float32, shape=(None, 3))
  area = compute_area(sides)
  result = sess.run(area, feed_dict = {sides: triangle_sides})
  print(result)

[6.4951906 4.419121  4.419121 ]


## tf.eager

tf.eager allows you to avoid the build-then-run stages. However, most production code will follow the lazy evaluation paradigm because the lazy evaluation paradigm is what allows for multi-device support and distribution. 
<p>
One thing you could do is to develop using tf.eager and then comment out the eager execution and add in the session management code.

<b>You may need to click on Reset Session to try this out.</b>

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def compute_area(sides):
  a = sides[:, 0]
  b = sides[:, 1]
  c = sides[:, 2]
  
  s = (a+b+c)/2
  
  return tf.sqrt(s*(s-a)*(s-b)*(s-c))

area = compute_area(tf.constant(triangle_sides))

print(area)

tf.Tensor([6.4951906 4.419121  4.419121 ], shape=(3,), dtype=float32)


## Challenge Exercise

Use TensorFlow to find the roots of a fourth-degree polynomial using [Halley's Method](https://en.wikipedia.org/wiki/Halley%27s_method).  The five coefficients (i.e. $a_0$ to $a_4$) of 
<p>
$f(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + a_4 x^4$
<p>
will be fed into the program, as will the initial guess $x_0$. Your program will start from that initial guess and then iterate one step using the formula:
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/142614c0378a1d61cb623c1352bf85b6b7bc4397" />
<p>
If you got the above easily, try iterating indefinitely until the change between $x_n$ and $x_{n+1}$ is less than some specified tolerance. Hint: Use [tf.while_loop](https://www.tensorflow.org/api_docs/python/tf/while_loop)

In [5]:
from time import time

def compute_fx(fx, x, order=1):
  for i in range(0, order):
    if grad not in locals():
      grad = tf.gradients(fx, x)[0]
    else:
      grad = tf.gradients(grad, x)[0]

  return grad

def next_iter(a, x, xn):
  a0 = a[:, 0]
  a1 = a[:, 1]
  a2 = a[:, 2]
  a3 = a[:, 3]
  a4 = a[:, 4]
  
  fx = a0 + a1*x + a2*x*x + a3*x*x*x + a4*x*x*x*x
  
  fx_1 = compute_fx(fx, x)
  fx_2 = compute_fx(fx, x, order=2)
  
  xn = x - (2*fx*fx_1)/(2*tf.pow(fx_1,2) - fx*fx_2)
  
  return [a, x, xn]

def stopper(x, xn, thres=0.0001):
  comp = tf.less(tf.sqrt(tf.squared_difference(xn, x)), thres)
  
  return tf.less(tf.reduce_sum(tf.cast(comp, tf.float32)), x.get_shape()[0].value)

def initialise(mean=0, var=1, shape=[2,5]):
  return tf.random_normal(shape, mean, var)

In [ ]:
tic = time()
with tf.Session() as sess:
  a = initialise()
  x = initialise(shape=[2])
  xn = initialise(mean=100, var=100, shape=[2])
  
  loop = tf.while_loop(stopper, next_iter, loop_vars=[a,x,xn], maximum_iterations=10)
  result = sess.run(loop)
  
  print('a: ',result[0])
  print('x: ',result[1])
  print('xn: ',result[2])

  print(time()-tic)

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License